<a href="https://colab.research.google.com/github/cagBRT/Dash/blob/main/Dash_Callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adding Callbacks to a Dash App

In [ ]:
!pip install jupyter-dash

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
from dash import html
from dash import dcc

We use html to create the layout for the app

# Example 1
This first example asks for an input from a text input box and then prints that text to the web page

In [ ]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H6("Change the value in the text box to see callbacks in action!"),
    html.Div([
        "Input: ",
        dcc.Input(id='my-input', value='initial value', type='text')
    ]),
    html.Br(),
    html.Div(id='my-output'),

])

Components in Dash usually update through user interaction like selecting a dropdown, dragging a slider, or hovering over points.



**Callbacks in Dash:** <br>
Making it interactive
Now, let’s have a look at how we can create the callback that will connect the dropdown and the stock prices line chart.

A callback is initialised using @app.callback(), which is followed by a function definition. Within this function, we define what happens on changing the value of the dropdown.<br>


Whenever the input value changes, the callback wrapper gets called automatically.
<br>
Component_id and component_property are keywords and optional in a callback. We saw this with the "Plotly 4 Dash" notebook. We did not have any inputs or outputs in the app, so we did not need the component_id or component_property. 

Notice how we don't set a value for the children property of the my-output component in the layout. When the Dash app starts, it automatically calls all of the callbacks with the initial values of the input components in order to populate the initial state of the output components.

In [ ]:
# Define callback inputs and outputs
@app.callback(
    Output(component_id='my-output', component_property='children'),
    Input(component_id='my-input', component_property='value')
)
def update_output_div(input_value):
    return f'Output: {input_value}'


In [ ]:
# Run app and display result inline in the notebook
app.run_server(mode='inline')